<img src="images/steno3d.png" width="200"></img>
<br>
<h1 style="color:#000080;"> Surface Example: Topography </h1>

This example shows how to plot gridded <a href="https://en.wikipedia.org/wiki/Digital_elevation_model">Digital Elevation Model (DEM)</a> surface in Steno3D. Figure 1 shows the elevation of a ridge (left) and the satellite picture of the ridge (right).

<div>
    <div style="height: 250px;">
    <img src="images/topo1.png" style="float: left; width: 50%; margin-top: 5%">
    <img src="images/topo2.png" style="float: right; width: 50%; margin-top: 5%">
    </div>
    <div style="width: 90%; margin-left: auto; margin-right: auto; font-weight: bold; font-style: italic; font-size: 15px; margin-top: 12%; margin-bottom: 3%"> Figure 1: Visualization of Topography in Steno3D. Left: elevation data on the surface. Right: texture (satellite picture) added to the surface </div>
</div>

<h3 style="color:#000080;"> Import Dependencies </h3>

In [ ]:
import json
import numpy as np
import steno3d

<h3 style="color:#000080;"> Login to Steno3D </h3> 

Before using Steno3D, you need to <a href="https://steno3d.com/signup">sign up for an account</a> to get your own developer API key if you do not have one already. Running the cell below will provide you with instructions for how to obtain and enter your key. 

The option to `skip_credentials` prevents saving your developer key to the mybinder.org server. When working on your local computer, remove this argument and your key will be saved.

In [ ]:
steno3d.login(skip_credentials=True)

<h3 style="color:#000080;"> Steno3D Surface </h3>

In this section we briefly describe the process of drawing surfaces in Steno3D before making the actual example. The way that Steno3D works is to first create a mesh and then bind the data to the mesh. The mesh defines the geometry of the surface. So the first step to create surfaces is to make a 2D mesh. In Steno3D, there are two types of meshes available for surfaces:

<ul>
    <li><b>Surface Mesh2D</b> (surfaces with triangulated structure)</li>
    <li><b>Surface Mesh2DGrid</b> (surfaces with gridded structure)</li>
</ul>

In this example we are using `Mesh2DGrid` since we are dealing with a structured DEM grid. For more information on `Mesh2D` see the other <a href="./tutorial_surface_triangle.ipynb">surface tutorial</a>.

Surface `Mesh2DGrid`, consists of x and y grid spacing with origin vector (O) corresponding to the lower-left corner of the grid. This type of mesh is normally used for structured surfaces such as mathematical surfaces which can be plotted as Z=f(X,Y). A simple diagram of `Mesh2DGrid` is depicted in Figure 2: <b>h1</b> and <b>h2</b> are arrays containing the width of each consecutive cell in the <b>U</b> and <b>V</b> directions, respectively. <b>Z</b> is an array of vertex topography, where values are perpendicular offset distances from the U-V plane. <b>Z</b> should be defined a flat array, with each entry corresponding to the z-offset of a single vertex. It is stored in row-major (C-style) order (more info on <a href="https://en.wikipedia.org/wiki/Row-major_order">row-major order</a>). For instance, the row-major order for `Mesh2DGrid` in Figure 2 would be stored as:

<p>[x0y0, x1y0, x2y0, x3y0, x4y0, x5y0, x0y1, x1y1 ...... x4y4, x5y4]</p>

<img src="images/mesh2dgrid.png" width=400 height=200 align=middle>
<div style="width: 30%; margin: auto; font-weight: bold; font-style: italic; font-size: 15px; margin-bottom: 5%; margin-top: 3%"> Figure 2: Surface Mesh2DGrid </div>

The next step is to bind the data to the surface. To do this we simply use a dictionary containing the data `location` (nodes/vertices 'N' or cell centers/faces 'CC') and the `DataArray`. When binding the data to a surface with `Mesh2DGrid` we need to specify data order. In Steno3D, this is stored in row major (C-style) by default, but column-major ordering (Fortran-style) is also available (<a href="https://python.steno3d.com/en/latest/content/api/resources/data.html#resources-data">DataArray documentation</a>). An important thing to keep in mind when binding the data to the surface is to convert the data into a flattened array using numpy's flatten( ) function. For example, as a n x m array, you need to flatten it into a (nm) x 1 array. Depending in which order you are going to flatten your data, you use either flatten(order='C') or flatten(order='F').

Another feature for Steno3D surfaces is that we can map an image to a surface mesh. We refer to the images projected on a surface as `textures`. Unlike data, `textures` do not correspond to certain mesh locations but instead are defined using spatial coordinates. In this example, we add a `texture` to the surface. For more information on `textures` please check <a href="https://python.steno3d.com/en/latest/content/api/resources/texture.html">Texture API documentation</a>.

<h3 style="color:#000080;"> Create a Project </h3>

Before making any resources (points, lines, surfaces or volumes) in Steno3D, we need to create a `Project` to assign the resource to that project. Projects organize all your resources together and can contain a combination of resources; they can also be viewed and shared in Steno3D. 

Following are the properties of projects that you can modify if desired:

<ul>
    <li><b>title</b>: Title of the project</li>
    <li><b>description</b>: Description of the project</li>
    <li><b>public</b>: If True, the project is viewable by anyone online. If False, the project is private and only viewable by people you choose to share it with. Free and Academic accounts have limits on the number of private projects allowed; if you would like more, upgrade your account at steno3d.com.</li>
    <li><b>resources</b>: List of resources the project contains.</li>
</ul>

You can <a href="https://steno3d.com/explore">explore public projects on steno3d.com.</a> Additional <a href="https://python.steno3d.com/en/latest/content/api/projects.html">API documentation</a> is available online.

In [ ]:
topography_proj = steno3d.Project(
    title='Topography',
    description='Digital Elevation Model (DEM)',
    public=False
)

<h3 style="color:#000080;"> Topography Data </h3>

In this section we are reading the topography data. The raw data are saved in JSON format which can be downloaded through the steno3d.examples.Topography module. This module downloads the data files and save it to a folder in the home directory (or another location you may specify). 

In [ ]:
from steno3d.examples import Topography
Topography.fetch_data(verbose=False)  # To specify a different data directory than the
                                      # default '$HOME/.steno3d_python_assets/' use
                                      # keyword argument `directory='/path/to/your/directory/'`

In [ ]:
with open(Topography.gridfile, 'r') as f:
    topo = json.load(f)

<h3 style="color:#000080;"> Surface Mesh2DGrid </h3>

Now it is time to create a `Mesh2DGrid`. Following are the properties of `Mesh2DGrid`:

<ul>
    <li><b>h1</b> and <b> h2 </b>(required)<sup><b>*</b></sup>: X and Y grid spacing</li>
    <li><b>O</b>: origin vector corresponding to the lower-left corner of the grid</li>
    <li><b>Z</b>: mesh vertex topography (perpendicular vector to the surface), in C-style, row-major order </li>
    <li><b>opts</b>: The only available option is `wireframe`, set to `True` or `False` with a dictionary </li>
    <li><b>title</b></li>
    <li><b>description</b></li>
</ul>

(*): For the Surface resource to pass validation prior to upload, required properties must be set.

Mesh2DGrid API documentation is available within the <a href="https://python.steno3d.com/en/latest/content/api/resources/surface.html#meshes">surface documentation</a>.

In [ ]:
topography_mesh=steno3d.Mesh2DGrid(
    h1=np.diff(topo['x']),   
    h2=np.diff(topo['y']),
    O=[topo['x'][0], topo['y'][0], 0],
    Z=topo['z']
)

<h3 style="color:#000080;"> Create Surface </h3>
 
The following are surface properties in Steno3D:

<ul>
    <li><b>project</b> (required): The project(s) that contain the resource</li>
    <li><b>mesh</b> (required): Surfaces may use `Mesh2D` or `Mesh2DGrid`</li>
    <li><b>data</b>: A list of `DataArrays` bound to the mesh </li>
    <li><b>textures</b>: A list of `Texture2DImages` </li>
    <li><b>opts</b>: A dictionary of options </li>
    <li><b>title</b></li>
    <li><b>description</b></li>
</ul>

Additional <a href="https://python.steno3d.com/en/latest/content/api/resources/surface.html">API documentation</a> is available online.

In [ ]:
topography_surface = steno3d.Surface(     
    project=topography_proj,
    title='DEM Surface',
    mesh=topography_mesh,
    opts=dict(opacity=1)
)

<h3 style="color:#000080;"> Binding Surface Data </h3>

Now that we have surface mesh, we can bind the data to it. To do this, we need a `DataArray` which contains:

<ul>
    <li><b>array</b> (required): 1D-array of data</li>
    <li><b>order</b>: Either 'c', C-style row-major ordering (default), or 'f', Fortran-style column-major ordering. This only applies when the data is bound to a Mesh2DGrid</li>
    <li><b>title</b></li>
    <li><b>description</b></li>
</ul>


Now we need to bind 1D `DataArray` to our surface with a dictionary containing:

<ul>
<li><b>location</b> (required): For surfaces, this is 'N' (nodes or vertices) or 'CC' (cell centers or faces/triangles)</li>
<li><b>data</b> (required): the DataArray</li>
</ul>

Also, more than one data set can be bound to a surface by using a list of these dictionaries. 

For a surface using `Mesh2DGrid`, the order of the data is decided based on order, either 'c' for C-style row-major ordering or 'f' for Fortran-style column-major ordering as it has been described in <b>Steno3D Surface</b> section of this document.

More information on binding data to a surface is available <a href="https://python.steno3d.com/en/latest/content/api/resources/surface.html#data">online</a>.

In [ ]:
topography_data = steno3d.DataArray(
    title='Topography',
    array=topo['z']
)

topography_surface.data = [
    dict(
        location='N',
        data=topography_data
    )    
]

<h3 style="color:#000080;"> Surface Textures </h3>

We can add textures to surfaces or points in Steno3D. Textures contain:

<ul>
    <li><b>image</b> (required)</li>
    <li><b>O</b> (required): the origin vector - this sets the lower-left corner of the image in space </li>
    <li><b>U</b> and <b>V</b> (required): the axes vectors - these determine the length and the direction of the sides of the image relative to O</li>
</ul>

More information on how to map an image to a point or surface resource is available on <a href="https://python.steno3d.com/en/latest/content/api/resources/texture.html">Texture</a>.

In [ ]:
my_tex = steno3d.Texture2DImage(
    image= Topography.topo_image,
    O=[443200.0, 491750.0, 0.0],
    U=[4425.0, 0.0, 0.0],
    V=[0.0, 3690.0, 0.0],
)
    
topography_surface.textures = my_tex

<h3 style="color:#000080;"> Upload and Plot </h3>

Calling `upload` will check that all aspects of surface and its containing project are built correctly, then upload the project to <a href="https://steno3d.com">steno3d.com</a>.

In [ ]:
topography_proj.upload()

In [ ]:
topography_surface.plot()